In [10]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qsl, urlunparse, urlencode, parse_qs
import os 


In [ ]:
import time 
for i in range(100):
	if i % 10 == 0:
		print('sleep 5s')
		time.sleep(5)
	else:
		print(i)

In [2]:
url = "https://trafficdata.tii.ie/tfmonthreport.asp?sgid=XzOA8m4lr27P0HaO3_srSB&spid=256365229484&reportdate=2021-10-27&enddate=2021-10-27"
document = requests.get(url, timeout=30)
html_doc = document.content
soup = BeautifulSoup(html_doc, 'html.parser')



In [40]:
import validators
site_eles = soup.select("ul.dropdown-menu li>a")
res = []
for ele in site_eles:
	href = ele['href']
	if validators.url(href):
		parsed_url = urlparse(href)
		query = dict(parse_qsl(parsed_url.query))
		query.pop('reportdate', None)
		query.pop('enddate', None)
		new_query = urlencode(query)
		new = parsed_url._replace(query=new_query)
		newUrl = urlunparse(new)
		res.append(newUrl)

with open('./list-url.txt', 'w') as f:
	for i in res:
		f.write(i)
		f.write('\n')


In [41]:
len(res)

376

In [42]:
len(list(set(res)))

376

In [39]:
from requests.models import PreparedRequest
url = "https://trafficdata.tii.ie/tfmonthreport.asp?sgid=XzOA8m4lr27P0HaO3_srSB&spid=256365229484&reportdate=abc"
parsed_url = urlparse(url)
query = dict(parse_qsl(parsed_url.query))
query.pop('reportdate', None)
query.pop('enddate', None)
new_query = urlencode(query)
new = parsed_url._replace(query=new_query)
newUrl = urlunparse(new)
print(newUrl)
# req = PreparedRequest()
# req.prepare_url(url, params)
# print(req.url)




https://trafficdata.tii.ie/tfmonthreport.asp?sgid=XzOA8m4lr27P0HaO3_srSB&spid=256365229484


In [16]:
def _get_siteId(soup):
	siteId_ele = soup.select('div#content #subtitlebar span')[1]
	siteId_text = siteId_ele.text.split(":")[1].strip()
	siteId_text = str(int(siteId_text))
	return siteId_text


In [18]:
def _get_monthly_data(soup):
	list_tr_element = soup.select("table#gridTable tr")
	content = []
	for i in range(1, len(list_tr_element)):
		tr_element = list_tr_element[i]
		if i == 1:
			# header 
			ths = tr_element.select('th')
			for j in range(len(ths)):
				content.append([ths[j].text])
		else:
			tds = tr_element.select('td')
			for j in range(len(tds)):
				content[j].append(tds[j].text)

	# convert content to pandas dataframe
	df = pd.DataFrame(content)
	df = df.transpose()
	new_header = df.iloc[0]
	df = df[1:]
	df.columns = new_header

	df.index = df['Time']
	df = df.drop(columns=['Time'], axis=1)
	return df

In [20]:

SAVE_FOLDER = os.path.join(os.getcwd(), "craw_data")
def get_montly_report(url):
	parsed_url = urlparse(url)
	query = parse_qs(parsed_url.query)
	report_date = query['reportdate'][0]
	document = requests.get(url, timeout=30)
	html_doc = document.content
	soup = BeautifulSoup(html_doc, 'html.parser')
	siteId_text = _get_siteId(soup)
	df = _get_monthly_data(soup)
	# save it to csv file 
	outfile_folfer = os.path.join(SAVE_FOLDER, report_date)
	if not os.path.exists(outfile_folfer):
		os.makedirs(outfile_folfer)
	df.to_csv(os.path.join(outfile_folfer, siteId_text + ".csv"))

url = "https://trafficdata.tii.ie/tfmonthreport.asp?sgid=XzOA8m4lr27P0HaO3_srSB&spid=256365229484&reportdate=2021-10-27&enddate=2021-10-27"
get_montly_report(url)


In [43]:
import random 
random.randint(5, 60)

12

In [1]:
!pip install yfinance

     |████████████████████████████████| 6.8 MB 6.1 MB/s 
Using legacy 'setup.py install' for yfinance, since package 'wheel' is not installed.
Using legacy 'setup.py install' for multitasking, since package 'wheel' is not installed.
    Running setup.py install for multitasking ... done
    Running setup.py install for yfinance ... done
You should consider upgrading via the '/home/minhnhat/python-env/analyse-movement/bin/python3 -m pip install --upgrade pip' command.


In [5]:
res = list(range(11))
n = 11
num_parts = n // 5 + 1
for part in range(num_parts):
	for i in res[part*5: (part + 1)*5]:
		print(i, " ")
	print('\n')

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [2]:
import yfinance as yf
stock_list = ['0005.HK', '0006.HK', '0066.HK', '0700.HK', '2800.HK']
print('stock_list:', stock_list)
data = yf.download(stock_list, start="2020-01-01", end="2020-02-21")
print('data fields downloaded:', set(data.columns.get_level_values(0)))
data.head()

ModuleNotFoundError: No module named 'yfinance'